# Segmentation 3D

## 1. Prerequisites

### 1.1 Load cloud point of voxel centers

In [1]:
import cv2

from openalea.phenomenal.data_access import (
    plant_1_voxel_grid,
    plant_2_voxel_grid,
    plant_1_images_binarize)

from openalea.phenomenal.display import *
from openalea.phenomenal.data_structure import (
    VoxelSkeleton,
    VoxelGrid,
    VoxelSegmentation,
    ImageView)

from openalea.phenomenal.multi_view_reconstruction import *

from openalea.phenomenal.segmentation_3D import (
    voxel_graph_from_voxel_grid,
    skeletonize,
    labelize_maize_skeleton,
    maize_analysis,
    segment_reduction)

from openalea.phenomenal.data_access.plant_1 import (
    plant_1_images_binarize,
    plant_1_calibration_camera_side,
    plant_1_calibration_camera_top)


In [2]:
def routine_select_ref_angle(image_views):

    max_len = 0
    image_view_max = None
    for i, iv in enumerate(image_views):

        x_pos, y_pos, x_len, y_len = cv2.boundingRect(cv2.findNonZero(iv.image))

        if x_len > max_len:
            max_len = x_len
            image_view_max = iv

    image_view_max.ref = True

    return image_views

In [3]:
images = plant_1_images_binarize()

calibration_side = plant_1_calibration_camera_side()
calibration_top = plant_1_calibration_camera_top()

# Select images
image_views = list()
for angle in range(0, 360, 30):
    projection = calibration_side.get_projection(angle)
    image_views.append(ImageView(images[angle], projection, inclusive=False))

image_views = routine_select_ref_angle(image_views)

projection = calibration_top.get_projection(0)
image_views.append(ImageView(images[-1], projection, inclusive=True))

voxels_size = 8
error_tolerance = 1
vg = reconstruction_3d(image_views,
                       voxels_size=voxels_size,
                       error_tolerance=error_tolerance,
                       verbose=True)

Iteration 1 / 9 : 8
Iteration 2 / 9 : 64
Iteration 3 / 9 : 512
Iteration 4 / 9 : 4096
Iteration 5 / 9 : 400
Iteration 6 / 9 : 1392
Iteration 7 / 9 : 4216
Iteration 8 / 9 : 13776
Iteration 9 / 9 : 55560


# Display voxel grid

In [4]:
from openalea.phenomenal.display.notebook import show_voxel_grid

show_voxel_grid(vg)

VBox(children=(Figure(anglex=1.5707963267948966, camera_center=[0.0, 0.0, 0.0], height=800, matrix_projection=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], matrix_world=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], scatters=[Scatter(color=array('green',
      dtype='|S5'), color_selected=array('white',
      dtype='|S5'), geo=u'box', size=array(1), size_selected=array(2.6), x=array([ 452.,  460.,  452., ...,  444.,  444.,  460.]), y=array([ 324.,  316.,  332., ...,  316.,  316.,  316.]), z=array([ -60.,  -60.,  -60., ...,  156.,  172.,  140.]))], style={'box': {'visible': False}, 'axes': {'color': 'black', 'visible': False, 'ticklabel': {'color': 'black'}, 'label': {'color': 'black'}}, 'background-color': 'white'}, tf=None, width=800, xlim=[-468.0, 524.0], ylim=[-444.0, 380.0], zlim=[-444.0, 748.0]),))

In [5]:
import time
t0 = time.time()

vg.voxels_position = map(tuple, list(vg.voxels_position))
vg.voxels_size = int(vg.voxels_size)

subgraph = None
voxel_graph = voxel_graph_from_voxel_grid(vg, connect_all_point=True)
voxel_skeleton = skeletonize(voxel_graph.graph,
                             voxel_graph.voxels_size,
                             subgraph=subgraph)

# Display it

In [6]:
from openalea.phenomenal.display.notebook import show_skeleton

show_skeleton(voxel_skeleton)

VBox(children=(Figure(anglex=1.5707963267948966, camera_center=[0.0, 0.0, 0.0], height=800, matrix_projection=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], matrix_world=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], scatters=[Scatter(color=array('green',
      dtype='|S5'), color_selected=array('white',
      dtype='|S5'), geo=u'box', size=array(0.25), size_selected=array(2.6), x=array([   4.,  -12., -124., ...,   12.,  -60.,   44.]), y=array([  -4.,   -4., -124., ...,   12.,  -44.,  -20.]), z=array([  20.,  452.,  596., ...,  404.,  484.,  468.])), Scatter(color_selected=array('white',
      dtype='|S5'), geo=u'box', size=array(1.0), size_selected=array(2.6), x=array([   4.,    4.,    4.,    4.,    4.,    4.,    4.,    4.,    4.,
          4.,    4.,    4.,    4.,    4.,    4.,    4.,    4.,    4.,
          4.,    4.,    4.,    4.,    4.,    4.,    4.,    4.,    4.,
          4.,    4.,    4.,    4.,    4.,    4.,    4.,    4.,    4.,
          4.,    4.,    4.,    4.,    4.,    4.,    4.,    4.,    4.,
          4.,    4.,    4.,    4.,    4.,    4.,    4.,    4.,    4.,
          4.,    4.,    4.,    4.,    4.,    4.,    4.,    4.,    4.,
          4.,    4.,    4.,    4.,    4.,    4.,    4.,    4.,    4.,
          4.,    4.,    4.,    4.,    4.,    4.,    4.,    4.,    4.,
          4.,    4.,    4.,    4.,    4.,    4.,    4.,    4.,    4.,
          4.,    4.,    4.,    4.,    4.,    4.,    4.,    4.,    4.,
          4.,    4.,    4.,    4.,    4.,    4.,    4.,    4.,    4.,
         -4.,  -12.,  -20.,  -20.,  -28.,  -28.,  -36.,  -36.,  -44.,
        -52.,  -52.,  -60.,  -68.,  -76.,  -84.,  -84.,  -92.,  -92.,
       -100., -108., -116., -124., -132., -140., -148., -156., -164.,
       -172., -180., -188., -196., -204., -212., -220., -228., -236.,
       -244., -252., -260., -268., -276., -284., -292., -300., -308.,
       -316., -324., -332., -340., -348., -356., -364., -372., -380.,
       -388., -396., -404., -412., -420., -428., -428., -428., -436.,
       -436., -436., -444., -444., -444., -444., -444., -428., -428.,
       -428., -428., -428., -420., -420., -420., -412., -412., -412.,
       -412., -412.]), y=array([  -4.,   -4.,   -4.,   -4.,   -4.,   -4.,   -4.,   -4.,   -4.,
         -4.,   -4.,   -4.,   -4.,   -4.,   -4.,   -4.,   -4.,   -4.,
         -4.,   -4.,   -4.,   -4.,   -4.,   -4.,   -4.,   -4.,   -4.,
         -4.,   -4.,   -4.,   -4.,   -4.,   -4.,   -4.,   -4.,   -4.,
         -4.,   -4.,   -4.,   -4.,   -4.,   -4.,   -4.,   -4.,   -4.,
         -4.,   -4.,   -4.,   -4.,   -4.,   -4.,   -4.,   -4.,  -12.,
        -12.,  -12.,  -12.,  -12.,  -12.,  -12.,  -12.,  -12.,  -12.,
        -12.,  -12.,  -12.,  -12.,  -12.,  -12.,  -12.,  -12.,  -12.,
        -12.,  -12.,  -12.,  -12.,  -12.,  -12.,  -12.,  -12.,  -12.,
        -12.,  -12.,  -12.,  -12.,  -12.,  -12.,  -12.,  -12.,  -12.,
        -12.,  -12.,  -12.,  -12.,  -12.,  -12.,  -12.,  -12.,  -12.,
        -12.,  -12.,  -12.,  -12.,  -12.,  -12.,  -12.,  -12.,  -12.,
        -20.,  -28.,  -36.,  -36.,  -44.,  -44.,  -44.,  -44.,  -44.,
        -44.,  -44.,  -44.,  -44.,  -44.,  -44.,  -44.,  -52.,  -52.,
        -60.,  -68.,  -76.,  -76.,  -84.,  -92., -100., -108., -116.,
       -124., -132., -140., -148., -148., -156., -156., -156., -156.,
       -156., -156., -164., -172., -180., -188., -196., -204., -212.,
       -212., -212., -220., -228., -236., -244., -252., -260., -268.,
       -276., -284., -292., -292., -300., -308., -308., -316., -324.,
       -324., -332., -340., -340., -348., -356., -364., -372., -372.,
       -372., -372., -372., -380., -380., -380., -388., -388., -388.,
       -388., -388.]), z=array([-444., -436., -428., -420., -412., -404., -396., -388., -380.,
       -372., -364., -356., -348., -340., -332., -324., -316., -308.,
       -300., -292., -284., -276., -268., -260., -252., -244., -236.,
       -228., -220., -212., -204., -196., -188., -180., -1

# Voxel segment reduction

In [7]:
# Select images
image_views = list()
for angle in range(0, 360, 30):
    projection = calibration_side.get_projection(angle)
    image_views.append(ImageView(images[angle],
                                 projection,
                                 inclusive=False))

voxel_skeleton = segment_reduction(voxel_skeleton,
                                   image_views,
                                   tolerance=4)

In [8]:
from openalea.phenomenal.display.notebook import show_skeleton

show_skeleton(voxel_skeleton, color="green")

VBox(children=(Figure(anglex=1.5707963267948966, camera_center=[0.0, 0.0, 0.0], height=800, matrix_projection=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], matrix_world=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], scatters=[Scatter(color=array('green',
      dtype='|S5'), color_selected=array('white',
      dtype='|S5'), geo=u'box', size=array(0.25), size_selected=array(2.6), x=array([ 116.,  132.,   12.,   36.,  148.,   76.,   28.,  132.,  108.,
         12.,   12.,  132.,   68.,   52.,   28.,  116.,   20.,   28.,
          4.,  124.,   68.,   84.,   52.,    4.,  140.,   60.,  -12.,
         12.,  108.,   -4.,   28.,  -20.,   60.,  124.,   28.,   28.,
         -4.,   12.,    4.,  100.,   68.,    4.,   28.,  100.,   28.,
         28.,   12.,    4.,    4.,  100.,   84.,    4.,   36.,    4.,
         68.,  132.,  132.,   -4.,   28.,   76.,   -4.,   28.,   -4.,
        108.,   12.,   12.,   76.,   12.,   -4.,   -4.,    4.,   20.,
        140.,   60.,  148.,  -20.,   -4.,   52.,  148.,  -12.,   -4.,
         12.,   84.,  132.,   36.,   68.,    4.,   44.,  148.,  116.,
         92.,   44.,  100.,    4.,  100.,   76.,  124.,   36.,   36.,
         84.,   60.,  140.,   12.,   20.,  148.,   52.,   52.,  132.,
          4.,   92.,   92.,   92.,    4.,   68.,   60.,   52.,  -28.,
         44.,   -4.,   36.,   28.,   28.,   20.,    4.,   20.,   76.,
         52.,  116.,   60.,   84.,   12.,   44.,    4.,   20.,   20.,
        100.,   20.,   28.,   44.,    4.,    4.,   60.,   44.,   12.,
         -4.,    4.,   12.,   52.,   60.,    4.,   12.,   84.,    4.,
         36.,   28.,   12.,   12.,   68.,  132.,   12.,    4.,  -12.,
         92.,   76.,   76.,   52.,   60.,   44.,   12.,   12.,  116.,
         12.,    4.,   -4.,   20.,   12.,   20.,   12.,    4.,  -20.,
          4.,   28.,  108.,   12.,   76.,    4.,    4.,   84.,   36.,
          4.,   20.,   68.,  -20.,   36.,   12.,   20.,   20.,  -20.,
         20.,   20.,   12.,   20.,   -4.,   36.,   76.,   28.,   28.,
        148.,   20.,   12.,   44.,   20.,   28.,  116.,   36.,  100.,
         52.,   12.,  116.,   28.,  156.,    4.,   28.,   12.,   20.,
        140.,   28.,  -20.,    4.,   12.,   20.,  140.,   92.,  156.,
         36.,  116.,   20.,   12.,   12.,   12.,   68.,   28.,    4.,
         28.,  148.,   60.,   12.,    4.,   12.,  124.,  100.,   -4.,
         36.,   28.,   76.,   20.,  140.,    4.,   12.,  156.,   20.,
         68.,    4.,   92.,   36.,  -20.,   -4.,  100.,    4.,  124.,
         28.,   12.,  116.,   12.,   44.,   36.,   20.,   36.,   -4.,
        -12.,   20.,   92.,   36.,   20.,   28.,   20.,   36.,   -4.,
         60.,   60.,   36.,   36.,   28.,   12.,  -12.,   20.,  108.,
         68.,  124.,   76.,   52.,   20.,   -4.,    4.,   20.,    4.,
        -28.,   36.,   -4.,   68.,   28.,  100.,   12.,   20.,   84.,
          4.,   76.,  140.,   36.,  -28.,    4.,   -4.,    4.,   84.,
         -4.,   52.,   44.,   52.,    4.,   92.,   20.,  148.,   44.,
        -20.,  108.,  124.,   52.,   28.,   12.,   20.,   -4.,   -4.,
          4.,   60.,   84.,  124.,   28.,   44.,   28.,   20.,  -12.,
         20.,   92.,  124.,   12.,   20.,  108.,   52.,   12.,   84.,
         20.,  -12.,  -12.,   -4.,    4.,  132.,   92.,   68.,   28.,
         76.,  -12.,   20.,    4.,   20.,   60.,  132.,   20.,   -4.,
         28.,    4.,  -20.,   -4.,  -20.,  108.,   12.,   76.,  116.,
         12.,    4.,   68.,   -4.,   68.,   84.,   20.,   84.,  140.,
         12.,    4.]), y=array([ -84.,  -84.,  -12.,  -28., -100.,  -44.,   -4.,  -92.,  -84.,
        -20.,   -4., -100.,  -28.,  -52.,  -12., -100.,    4.,   -4.,
          4., -100.,  -36.,  -44.,  -28.,  -12., -100.,  -12.,   20.,
         -4.,  -84.,   -4.,  -20.,    4.,  -52.,  -92.,   -4.,  -20.,
         -4.,  -12.,  -12.,  -92.,  -36.,    4.,    4.,  -52.,   -4.,
         -4.,   -4.,  -12.,   -4.,  -84.,  -76.,  -12.,  -20.,    4.,
        -60., -100., 

# Segmentation Maize

In [9]:
vms = labelize_maize_skeleton(voxel_skeleton, voxel_graph)
filename = 'voxel_maize_segmentation.json'
vms.write_to_json_gz(filename)
vms = VoxelSegmentation.read_from_json_gz(filename)

In [10]:
from openalea.phenomenal.display.notebook import show_segmentation

show_segmentation(vms)

VBox(children=(Figure(anglex=1.5707963267948966, camera_center=[0.0, 0.0, 0.0], height=800, matrix_projection=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], matrix_world=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], scatters=[Scatter(color=array('rgb(128, 128, 128)',
      dtype='|S18'), color_selected=array('white',
      dtype='|S5'), geo=u'box', size=array(1.0), size_selected=array(2.6), x=array([  4.,  20.,   4., ...,  12.,  12.,  12.]), y=array([-20., -12., -20., ..., -20.,   4., -20.]), z=array([-172.,   84., -108., ...,  -28., -372., -396.])), Scatter(color=array('rgb(255, 0, 0)',
      dtype='|S14'), color_selected=array('white',
      dtype='|S5'), geo=u'box', size=array(1.0), size_selected=array(2.6), x=array([-68.,  52.,  36., ..., -28.,  76.,  20.]), y=array([ -92.,   12.,   -4., ..., -100.,  -20.,   68.]), z=array([ 636.,  364.,  372., ...,  596.,  460.,  548.])), Scatter(color=array('rgb(255, 0, 0)',
      dtype='|S14'), color_selected=array('white',
      dtype='|S5'), geo=u'box', size=array(1.0), size_selected=array(2.6), x=array([ 52.,  36., -92., ..., -28.,  76.,  20.]), y=array([  12.,   -4.,  -60., ..., -100.,  -20.,   68.]), z=array([ 364.,  372.,  516., ...,  596.,  460.,  548.])), Scatter(color=array('rgb(255, 0, 0)',
      dtype='|S14'), color_selected=array('white',
      dtype='|S5'), geo=u'box', size=array(1.0), size_selected=array(2.6), x=array([ 92.,  36.,   4., ...,  60.,  76.,  68.]), y=array([ 36.,  -4., -60., ...,  44., -20., -76.]), z=array([ 380.,  372.,  476., ...,  484.,  460.,  548.])), Scatter(color=array('rgb(255, 0, 0)',
      dtype='|S14'), color_selected=array('white',
      dtype='|S5'), geo=u'box', size=array(1.0), size_selected=array(2.6), x=array([ 124.,   36.,  140., ...,  196.,  -44.,  220.]), y=array([ 124.,   -4.,  108., ...,   84.,   20.,  116.]), z=array([ 596.,  372.,  620., ...,  532.,  468.,  588.])), Scatter(color=array('rgb(255, 0, 0)',
      dtype='|S14'), color_selected=array('white',
      dtype='|S5'), geo=u'box', size=array(1.0), size_selected=array(2.6), x=array([  52.,  124.,   36., ...,   52.,  -12.,  -20.]), y=array([ -52.,  124.,   -4., ...,   36.,  -60.,  -68.]), z=array([ 380.,  596.,  372., ...,  532.,  420.,  548.])), Scatter(color=array('rgb(255, 0, 0)',
      dtype='|S14'), color_selected=array('white',
      dtype='|S5'), geo=u'box', size=array(1.0), size_selected=array(2.6), x=array([-220., -244.,  -76., ..., -388., -116., -196.]), y=array([-212., -292., -100., ..., -380., -124., -276.]), z=array([ 476.,  468.,  356., ...,  396.,  396.,  452.])), Scatter(color=array('rgb(0, 255, 0)',
      dtype='|S14'), color_selected=array('white',
      dtype='|S5'), geo=u'box', size=array(1.0), size_selected=array(2.6), x=array([ 428.,  244.,   76., ...,  460.,  284.,   92.]), y=array([ 316.,  196.,   44., ...,  332.,  212.,   12.]), z=array([ 172.,  276.,  172., ...,   44.,  268.,  132.])), Scatter(color=array('rgb(0, 255, 0)',
      dtype='|S14'), color_selected=array('white',
      dtype='|S5'), geo=u'box', size=array(1.0), size_selected=array(2.6), x=array([-308.,   -4., -244., ...,  -68., -380., -188.]), y=array([-324.,  -20., -220., ...,  -84., -404., -188.]), z=array([  12., -100.,   60., ...,   -4., -180.,   52.])), Scatter(color=array('rgb(0, 255, 0)',
      dtype='|S14'), color_selected=array('white',
      dtype='|S5'), geo=u'box', size=array(1.0), size_selected=array(2.6), x=array([ 292.,  220.,   76.,  340.,  300.,  140.,  268.,  180.,  148.,
        156.,  108.,  252.,   28.,  172.,  308.,  300.,  140.,  340.,
        260.,   68.,  180.,  348.,  252.,  332.,  156.,  188.,  340.,
        124.,  332.,  196.,  268.,  196.,  188.,  316.,  188.,  148.,
        332.,  204.,  156.,  132.,  340.,  300.,  244.,  228.,  332.,
        148.,   44.,  156.,  348.,  228.,  100.,  204.,  284.,  292.,
         60.,  332.,  180.,  292.,  332.,  324.,  268.,   44.,  284.,
         52.,  236.,  260., 

In [11]:
vmsi = maize_analysis(vms)
file_prefix = 'voxel_maize_segmentation_info'
vmsi.write_to_json_gz(file_prefix)
vmsi = VoxelSegmentation.read_from_json_gz(file_prefix)

In [15]:
show_segmentation(vms, size_ratio=0.5)

VBox(children=(Figure(anglex=1.5707963267948966, camera_center=[0.0, 0.0, 0.0], height=800, matrix_projection=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], matrix_world=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], scatters=[Scatter(color=array('rgb(128, 128, 128)',
      dtype='|S18'), color_selected=array('white',
      dtype='|S5'), geo=u'box', size=array(0.5), size_selected=array(2.6), x=array([  4.,  20.,   4., ...,  12.,  12.,  12.]), y=array([-20., -12., -20., ..., -20.,   4., -20.]), z=array([-172.,   84., -108., ...,  -28., -372., -396.])), Scatter(color=array('rgb(93, 154, 193)',
      dtype='|S17'), color_selected=array('white',
      dtype='|S5'), geo=u'box', size=array(0.5), size_selected=array(2.6), x=array([-68.,  52.,  36., ..., -28.,  76.,  20.]), y=array([ -92.,   12.,   -4., ..., -100.,  -20.,   68.]), z=array([ 636.,  364.,  372., ...,  596.,  460.,  548.])), Scatter(color_selected=array('white',
      dtype='|S5'), geo=u'sphere', size=array(1.0), size_selected=array(2.6), x=array([-52.]), y=array([-260.]), z=array([ 644.])), Scatter(color=array('blue',
      dtype='|S4'), color_selected=array('white',
      dtype='|S5'), geo=u'sphere', size=array(1.0), size_selected=array(2.6), x=array([ 4.]), y=array([-76.]), z=array([ 516.])), Scatter(color=array('rgb(126, 47, 163)',
      dtype='|S17'), color_selected=array('white',
      dtype='|S5'), geo=u'box', size=array(0.5), size_selected=array(2.6), x=array([ 52.,  36., -92., ..., -28.,  76.,  20.]), y=array([  12.,   -4.,  -60., ..., -100.,  -20.,   68.]), z=array([ 364.,  372.,  516., ...,  596.,  460.,  548.])), Scatter(color_selected=array('white',
      dtype='|S5'), geo=u'sphere', size=array(1.0), size_selected=array(2.6), x=array([ 20.]), y=array([-180.]), z=array([ 644.])), Scatter(color=array('blue',
      dtype='|S4'), color_selected=array('white',
      dtype='|S5'), geo=u'sphere', size=array(1.0), size_selected=array(2.6), x=array([ 28.]), y=array([-116.]), z=array([ 652.])), Scatter(color=array('rgb(25, 243, 129)',
      dtype='|S17'), color_selected=array('white',
      dtype='|S5'), geo=u'box', size=array(0.5), size_selected=array(2.6), x=array([ 92.,  36.,   4., ...,  60.,  76.,  68.]), y=array([ 36.,  -4., -60., ...,  44., -20., -76.]), z=array([ 380.,  372.,  476., ...,  484.,  460.,  548.])), Scatter(color_selected=array('white',
      dtype='|S5'), geo=u'sphere', size=array(1.0), size_selected=array(2.6), x=array([-412.]), y=array([-388.]), z=array([ 420.])), Scatter(color=array('blue',
      dtype='|S4'), color_selected=array('white',
      dtype='|S5'), geo=u'sphere', size=array(1.0), size_selected=array(2.6), x=array([ 4.]), y=array([-12.]), z=array([ 140.])), Scatter(color=array('rgb(76, 197, 146)',
      dtype='|S17'), color_selected=array('white',
      dtype='|S5'), geo=u'box', size=array(0.5), size_selected=array(2.6), x=array([ 124.,   36.,  140., ...,  196.,  -44.,  220.]), y=array([ 124.,   -4.,  108., ...,   84.,   20.,  116.]), z=array([ 596.,  372.,  620., ...,  532.,  468.,  588.])), Scatter(color_selected=array('white',
      dtype='|S5'), geo=u'sphere', size=array(1.0), size_selected=array(2.6), x=array([ 492.]), y=array([ 356.]), z=array([ 340.])), Scatter(color=array('blue',
      dtype='|S4'), color_selected=array('white',
      dtype='|S5'), geo=u'sphere', size=array(1.0), size_selected=array(2.6), x=array([ 84.]), y=array([-4.]), z=array([ 396.])), Scatter(color=array('rgb(83, 163, 134)',
      dtype='|S17'), color_selected=array('white',
      dtype='|S5'), geo=u'box', size=array(0.5), size_selected=array(2.6), x=array([  52.,  124.,   36., ...,   52.,  -12.,  -20.]), y=array([ -52.,  124.,   -4., ...,   36.,  -60.,  -68.]), z=array([ 380.,  596.,  372., ...,  532.,  420.,  548.])), Scatter(color_selected=array('white',
      dtype='|S5'), geo=u'sphere', size=array(1.0), size_selected=array(2.6), x=array([ 204.]), y=array([ 196.]), z=array([ 652.])),